In [1]:
!git clone https://github.com/kimbold/VRDL_2019

Cloning into 'VRDL_2019'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 4147 (delta 103), reused 256 (delta 83), pack-reused 3865
Receiving objects: 100% (4147/4147), 55.84 MiB | 30.84 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
%cd VRDL_2019/HW3

/content/VRDL_2019/HW3


In [4]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1DryVeYEC5mlo0YFHltN6EhO4zFuTIvNn

Downloading...
From: https://drive.google.com/uc?id=1DryVeYEC5mlo0YFHltN6EhO4zFuTIvNn
To: /content/VRDL_2019/HW3/weights.h5
285MB [00:01, 151MB/s]


In [5]:
#Get test data as specified by hw3
!gdown https://drive.google.com/uc?id=1GV4TLocxFcsR8QPdM-rP8RoOB2vjs_E_

Downloading...
From: https://drive.google.com/uc?id=1GV4TLocxFcsR8QPdM-rP8RoOB2vjs_E_
To: /content/VRDL_2019/HW3/test.zip
272MB [00:02, 102MB/s]


In [6]:
!pip install tensorflow==1.14.0
!pip install keras==2.1.1
!pip install imgaug==0.2.6
!pip install opencv-python
!pip install Pillow
!pip install requests
!pip install tqdm
!pip install sklearn
!pip install pytest-cov
!pip install codecov
!pip install matplotlib

     |████████████████████████████████| 109.2MB 85kB/s 
     |████████████████████████████████| 3.2MB 38.0MB/s 
     |████████████████████████████████| 491kB 28.3MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 307kB 4.7MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.1 which is incompatible.
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
     |████████████████████████████████| 634kB 5.2MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.wh

**Load svhn dataset**

In [0]:
training=0

In [0]:
if(training==1):
  !wget http://ufldl.stanford.edu/housenumbers/train.tar.gz

#Get test data from vrdl google drive for hw3:

#!wget http://ufldl.stanford.edu/housenumbers/test.tar.gz

**Extract dataset files**

In [9]:
#!tar -zxvf test.zip
!unzip test.zip
# Comment training out
#if(training==1):
#  !tar -zxvf train.tar.gz

#!tar -zxvf test.tar.gz

Archive:  test.zip
   creating: test/
  inflating: test/1.png              
  inflating: test/10.png             
  inflating: test/100.png            
  inflating: test/1000.png           
  inflating: test/10000.png          
  inflating: test/10001.png          
  inflating: test/10002.png          
  inflating: test/10003.png          
  inflating: test/10004.png          
  inflating: test/10005.png          
  inflating: test/10006.png          
  inflating: test/10007.png          
  inflating: test/10008.png          
  inflating: test/10009.png          
  inflating: test/1001.png           
  inflating: test/10010.png          
  inflating: test/10011.png          
  inflating: test/10012.png          
  inflating: test/10013.png          
  inflating: test/10014.png          
  inflating: test/10015.png          
  inflating: test/10016.png          
  inflating: test/10017.png          
  inflating: test/10018.png          
  inflating: test/10019.png          
  inflating:

**Import required modules**

In [10]:
import numpy as np
import argparse
import os
import json
from yolo.frontend import create_yolo, get_object_labels
import xml.etree.cElementTree as ET
import h5py
from lxml import etree

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Using TensorFlow backend.


**Define methods to load annotations**

In [0]:
def get_name(index, hdf5_data):
    name = hdf5_data['/digitStruct/name']
    return ''.join([chr(v[0]) for v in hdf5_data[name[index][0]].value])

def keys(f):
    return [key for key in f.keys()]

def get_bbox(index, hdf5_data):
  """
  The box data contains width and height of the box as well as the upper left corner.
  By supplying the distance from the left border and top border, the upper left point can be identified.
  """
  attrs = {}
  item = hdf5_data['digitStruct']['bbox'][index].item()
  for key in ['label', 'left', 'top', 'width', 'height']:
      attr = hdf5_data[item][key]
      values = [hdf5_data[attr.value[i].item()].value[0][0]
                for i in range(len(attr))] if len(attr) > 1 else [attr.value[0][0]]
      attrs[key] = values
  return attrs

def prettyPrintXml(xmlFilePathToPrettyPrint):
    parser = etree.XMLParser(resolve_entities=False, strip_cdata=False)
    document = etree.parse(xmlFilePathToPrettyPrint, parser)
    document.write(xmlFilePathToPrettyPrint, pretty_print=True, encoding='utf-8')

# replacement strings
WINDOWS_LINE_ENDING = b'\r\n'
UNIX_LINE_ENDING = b'\n'

def replace_Unix_with_Windows_in_XLM(xmlFilePath):
    with open(xmlFilePath, 'rb') as open_file:
      content = open_file.read()

    content = content.replace(UNIX_LINE_ENDING,WINDOWS_LINE_ENDING)

    with open(xmlFilePath, 'wb') as open_file:
        open_file.write(content)

In [0]:
import os
directory = "tests/dataset/svhn/test_anns"
if not os.path.exists(directory):
    os.makedirs(directory)

**Load annotations for training**

In [0]:
if(training==1):
  # Need to define method to read .mat data and then write annotation files for each picture
  filename = "train/digitStruct.mat"

  f = h5py.File(filename,'r')

  # Iterate through all images to create the annotation file
  for image in range(f['digitStruct']['bbox'].shape[0]):
  #for image in range(3):
    #annotation
    root = ET.Element("annotation")

    #Get image annotation data
    size_data = get_bbox(image, f)

    #filename
    filename = get_name(image,f)
    doc = ET.SubElement(root, "filename").text = filename

    # For each detected digit, add the data for the box for it
    object_list = [[] for _ in range(len(size_data['label']))]
    bndbox = [[] for _ in range(len(size_data['label']))]

    for number in range(len(size_data['label'])):
      object_list[number] = ET.SubElement(root, "object")
      ET.SubElement(object_list[number], "name").text = str(int(size_data['label'][number]))

      bndbox[number] = ET.SubElement(object_list[number], "bndbox")
      ET.SubElement(bndbox[number], "xmin").text = str(int(size_data['left'][number]))
      ET.SubElement(bndbox[number], "ymin").text = str(int(size_data['top'][number]))
      ET.SubElement(bndbox[number], "xmax").text = str(int(size_data['width'][number]+size_data['left'][number]))
      ET.SubElement(bndbox[number], "ymax").text = str(int(size_data['height'][number]+size_data['top'][number]))
    
    tree = ET.ElementTree(root)
    tree.write("tests/dataset/svhn/anns/"+str(image+1)+".xml")
    prettyPrintXml("tests/dataset/svhn/anns/"+str(image+1)+".xml")
    replace_Unix_with_Windows_in_XLM("tests/dataset/svhn/anns/"+str(image+1)+".xml")

  if os.path.exists('tests/dataset/svhn/anns/.ipynb_checkpoints'):
    os.rmdir('tests/dataset/svhn/anns/.ipynb_checkpoints')

In [0]:
# Need to define method to read .mat data and then write annotation files for each picture
enabled=0
if(enabled==1):
  filename = "train/digitStruct.mat"

  f = h5py.File(filename,'r')

  # Iterate through all images to create the annotation file
  #for image in range(len(f['digitStruct']['bbox'].shape[0])):
  for image in [2,3]:
    #annotation
    root = ET.Element("annotation")

    #Get image annotation data
    size_data = get_bbox(image, f)

    #filename
    filename = get_name(image,f)
    doc = ET.SubElement(root, "filename").text = filename

    # For each detected digit, add the data for the box for it
    object_list = [[] for _ in range(len(size_data['label']))]
    bndbox = [[] for _ in range(len(size_data['label']))]

    for number in range(len(size_data['label'])):
      object_list[number] = ET.SubElement(root, "object")
      ET.SubElement(object_list[number], "name").text = str(int(size_data['label'][number]))

      bndbox[number] = ET.SubElement(object_list[number], "bndbox")
      ET.SubElement(bndbox[number], "xmin").text = str(int(size_data['left'][number]))
      ET.SubElement(bndbox[number], "ymin").text = str(int(size_data['top'][number]))
      ET.SubElement(bndbox[number], "xmax").text = str(int(size_data['width'][number]+size_data['left'][number]))
      ET.SubElement(bndbox[number], "ymax").text = str(int(size_data['height'][number]+size_data['top'][number]))

    
    tree = ET.ElementTree(root)
    tree.write("tests/dataset/svhn/test_anns/"+str(image+1)+".xml")
    prettyPrintXml("tests/dataset/svhn/test_anns/"+str(image+1)+".xml")
    replace_Unix_with_Windows_in_XLM("tests/dataset/svhn/test_anns/"+str(image+1)+".xml")
    
  if os.path.exists('tests/dataset/svhn/test_anns/.ipynb_checkpoints'):
    os.rmdir('tests/dataset/svhn/test_anns/.ipynb_checkpoints')

In [0]:
if(training==1):
  !dir tests/dataset/svhn/anns
  !dir tests/dataset/svhn/test_anns

**Train yolo**

Training yolo takes around 3 hours for all layers here
and then 2 more hours for the final layer.

In [0]:
#Train and validate YOLO_v2 model on any dataset
def setup_training(config_file):
    """make directory to save weights & its configuration """
    import shutil
    with open(config_file) as config_buffer:
        config = json.loads(config_buffer.read())
    dirname = config['train']['saved_folder']
    if os.path.isdir(dirname):
        print("{} is already exists. Weight file in directory will be overwritten".format(dirname))
    else:
        print("{} is created.".format(dirname, dirname))
        os.makedirs(dirname)
    print("Weight file and Config file will be saved in \"{}\"".format(dirname))
    shutil.copyfile(config_file, os.path.join(dirname, "config.json"))
    return config, os.path.join(dirname, "weights.h5")


def train(conf="configs/from_scratch.json"):
  #path to configuration file

    config, weight_file = setup_training(conf)
    
    if config['train']['is_only_detect']:
        labels = ["object"]
    else:
        if config['model']['labels']:
            labels = config['model']['labels']
        else:
            labels = get_object_labels(config['train']['train_annot_folder'])
    print(labels)

    # 1. Construct the model 
    yolo = create_yolo(config['model']['architecture'],
                       labels,
                       config['model']['input_size'],
                       config['model']['anchors'],
                       config['model']['coord_scale'],
                       config['model']['class_scale'],
                       config['model']['object_scale'],
                       config['model']['no_object_scale'])

    # 2. Load the pretrained weights (if any) 
    yolo.load_weights(config['pretrained']['full'], by_name=True)

    # 3. actual training 
    yolo.train(config['train']['train_image_folder'],
               config['train']['train_annot_folder'],
               config['train']['actual_epoch'],
               weight_file,
               config["train"]["batch_size"],
               config["train"]["jitter"],
               config['train']['learning_rate'], 
               config['train']['train_times'],
               config['train']['valid_times'],
               config['train']['valid_image_folder'],
               config['train']['valid_annot_folder'],
               config['train']['first_trainable_layer'],
               config['train']['is_only_detect'])
    # loss: 2.1691, train batch jitter=False


In [0]:
if(training==1):
  train(conf="configs/from_scratch_custom.json")

In [0]:
if(training==1):  
  train(conf="configs/from_scratch2_custom.json")

**After training the weights are stored in .h5 file**

So now it is necessary to ensure that it is closed properly to avoid corruption.

In [0]:
# Closing all .hd5 files: https://stackoverflow.com/questions/29863342/close-an-open-h5py-data-file against corruption
import tables
tables.file._open_files.close_all()

**Evaluate trained yolo digit detector**

In [20]:
import argparse
import json
import cv2
import numpy as np
from yolo.frontend import create_yolo
from yolo.backend.utils.box import draw_scaled_boxes
from yolo.backend.utils.annotation import parse_annotation
from yolo.backend.utils.eval._box_match import BoxMatcher

import os
import yolo

DEFAULT_CONFIG_FILE = os.path.join(yolo.PROJECT_ROOT, "svhn", "config.json")
DEFAULT_WEIGHT_FILE = os.path.join(yolo.PROJECT_ROOT, "svhn", "weights.h5")

DEFAULT_THRESHOLD = 0.3


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [0]:
def count_true_positives(detect_boxes, true_boxes, detect_labels=None, true_labels=None):
    """
    # Args
        detect_boxes : array, shape of (n_detected_boxes, 4)
        true_boxes : array, shape of (n_true_boxes, 4)
        detected_labels : array, shape of (n_detected_boxes,)
        true_labels :
    """
    n_true_positives = 0
    print("Detect boxes:",detect_boxes,"True boxes", true_boxes)
    matcher = BoxMatcher(detect_boxes, true_boxes, detect_labels, true_labels)
    for i in range(len(detect_boxes)):
        matching_idx, iou = matcher.match_idx_of_box1_idx(i)
        # print("detect_idx: {}, true_idx: {}, matching-score: {}".format(i, matching_idx, iou))
        if matching_idx is not None and iou > 0.5:
            n_true_positives += 1
    return n_true_positives


def calc_score(n_true_positives, n_truth, n_pred):
    """
    # Args
        detect_boxes : list of box-arrays
        true_boxes : list of box-arrays
    """
    print("True positives:", n_true_positives)
    print("n_pred:", n_pred)
    print("n_truth:", n_truth)
    precision = n_true_positives / n_pred
    recall = n_true_positives / n_truth
    print("Precision:", precision)
    print("Recall:", recall)
    fscore = 2* precision * recall / (precision + recall)
    score = {"fscore": fscore, "precision": precision, "recall": recall}
    return score

In [0]:
def evaluate(conf=DEFAULT_CONFIG_FILE, weights=DEFAULT_WEIGHT_FILE, threshold=DEFAULT_THRESHOLD):

      with open(conf) as config_buffer:
          config = json.loads(config_buffer.read())

      # 2. create yolo instance & predict
      yolo = create_yolo(config['model']['architecture'],
                        config['model']['labels'],
                        config['model']['input_size'],
                        config['model']['anchors'])
      yolo.load_weights(weights)

      # 3. read image
      write_dname = "detected"
      if not os.path.exists(write_dname): os.makedirs(write_dname)
      annotations = parse_annotation(config['train']['valid_annot_folder'],
                                    config['train']['valid_image_folder'],
                                    config['model']['labels'],
                                    is_only_detect=config['train']['is_only_detect'])

      n_true_positives = 0
      n_truth = 0
      n_pred = 0
      for i in range(len(annotations)):
          img_path = annotations.fname(i)
          img_fname = os.path.basename(img_path)
          image = cv2.imread(img_path)
          true_boxes = annotations.boxes(i)
          true_labels = annotations.code_labels(i)
          
          boxes, probs = yolo.predict(image, float(threshold))
          labels = np.argmax(probs, axis=1) if len(probs) > 0 else [] 
        
          # 4. save detection result
          image = draw_scaled_boxes(image, boxes, probs, config['model']['labels'])
          output_path = os.path.join(write_dname, os.path.split(img_fname)[-1])
          
          cv2.imwrite(output_path, image)
          print("{}-boxes are detected. {} saved.".format(len(boxes), output_path))

          n_true_positives += count_true_positives(boxes, true_boxes, labels, true_labels)
          n_truth += len(true_boxes)
          n_pred += len(boxes)
      print(calc_score(n_true_positives, n_truth, n_pred))

In [0]:
#evaluate()

In [0]:
#evaluate(weights="/content/VRDL_2019/HW3/svhn/weights.h5", threshold=0.3)

**Predict test data**

In [0]:
import json

def create_yolo_instance(conf=DEFAULT_CONFIG_FILE, weights=DEFAULT_WEIGHT_FILE):
  with open(conf) as config_buffer:
    config = json.loads(config_buffer.read())
  
  # 2. create yolo instance & predict
  yolo = create_yolo(config['model']['architecture'],
                     config['model']['labels'],
                     config['model']['input_size'],
                     config['model']['anchors'])
  yolo.load_weights(weights)
  return yolo


def predict_boxes_digits(image_number_start=10, image_number_end=10, image_folder="test/",conf=DEFAULT_CONFIG_FILE, weights=DEFAULT_WEIGHT_FILE, threshold=DEFAULT_THRESHOLD):

      # Create list of dictionaries for submission
      prediction_dictionaries = [{} for _ in range(image_number_end-image_number_start)]


      with open(conf) as config_buffer:
          config = json.loads(config_buffer.read())

      # 2. create yolo instance & predict
      yolo = create_yolo(config['model']['architecture'],
                        config['model']['labels'],
                        config['model']['input_size'],
                        config['model']['anchors'])
      yolo.load_weights(weights)

      # 3. read image
      write_dname = "detected"
      if not os.path.exists(write_dname): os.makedirs(write_dname)
  
      for i in range(image_number_start,image_number_end):
          # For each image, get the predicted labels, probabilities and boxes

          img_path=image_folder+str(i)+".png"
          print(img_path)
          img_fname = os.path.basename(img_path)
          image = cv2.imread(img_path)
          
          boxes, probs = yolo.predict(image, float(threshold))
          labels = np.argmax(probs, axis=1) if len(probs) > 0 else [] 
          print("Boxes:",boxes, "Type:",type(boxes))
          print("SCore:",[np.max(probs[i]) for i in range(len(probs))])
          print("Labels:",labels)

            
          try:
            prediction_dictionaries[i-1]['bbox']=boxes.tolist()
          except:
            prediction_dictionaries[i-1]['bbox']=boxes

          try:
            prediction_dictionaries[i-1]['label']=labels.tolist()
          except:
            prediction_dictionaries[i-1]['label']=labels

          prediction_dictionaries[i-1]['score']=[np.max(probs[i]) for i in range(len(probs))]

          print(prediction_dictionaries)
          # 4. save detection result
          image = draw_scaled_boxes(image, boxes, probs, config['model']['labels'])
          output_path = os.path.join(write_dname, os.path.split(img_fname)[-1])
          
          cv2.imwrite(output_path, image)
          print("{}-boxes are detected. {} saved.".format(len(boxes), output_path))
      return prediction_dictionaries
      #with open("0845058_"+str(image_number_start)+"_"+str(image_number_end)+"_.json", 'w') as f:
      #    f.write(json.dumps(prediction_dictionaries))
      #return boxes, probs, labels, prediction_dictionaries

def predict_testdata(image_folder="test/",conf=DEFAULT_CONFIG_FILE, weights=DEFAULT_WEIGHT_FILE, threshold=DEFAULT_THRESHOLD):

      # Create list of dictionaries for submission
      prediction_dictionaries = [{} for _ in range(1,13069)]


      with open(conf) as config_buffer:
          config = json.loads(config_buffer.read())

      # 2. create yolo instance & predict
      yolo = create_yolo(config['model']['architecture'],
                        config['model']['labels'],
                        config['model']['input_size'],
                        config['model']['anchors'])
      yolo.load_weights(weights)

      # 3. read image
      write_dname = "detected"
      if not os.path.exists(write_dname): os.makedirs(write_dname)
  
      for i in range(1,13069):
          # For each image, get the predicted labels, probabilities and boxes

          img_path=image_folder+str(i)+".png"
          print(img_path)
          img_fname = os.path.basename(img_path)
          image = cv2.imread(img_path)
          boxes = [[]]
          probs = []
          labels = []
          boxes, probs = yolo.predict(image, float(threshold))
          labels = np.argmax(probs, axis=1) if len(probs) > 0 else [] 
            
          try:
            prediction_dictionaries[i-1]['bbox']=boxes.tolist()
          except:
            prediction_dictionaries[i-1]['bbox']=boxes

          prediction_dictionaries[i-1]['score']=[np.max(probs[i]) for i in range(len(probs))]

          try:
            prediction_dictionaries[i-1]['label']=labels.tolist()
          except:
            prediction_dictionaries[i-1]['label']=labels

          #print(prediction_dictionaries)
          # 4. save detection result
          image = draw_scaled_boxes(image, boxes, probs, config['model']['labels'])
          output_path = os.path.join(write_dname, os.path.split(img_fname)[-1])
          
          cv2.imwrite(output_path, image)
          #print("{}-boxes are detected. {} saved.".format(len(boxes), output_path))
      with open("0845058.json", 'w') as f:
        f.write(str(prediction_dictionaries))

**Timing for one picture**

In [0]:
DEFAULT_WEIGHT_FILE='/content/VRDL_2019/HW3/weights.h5'

In [27]:
#Create yolo instance
#weights="/content/VRDL_2019/HW3/svhn/weights.h5"
conf = os.path.join(yolo.PROJECT_ROOT, "config_customized.json")
yolo = create_yolo_instance(conf=conf, weights=DEFAULT_WEIGHT_FILE)

#Load image from test folder (unseen)
image_folder="test/"
i=1 # load the first image
img_path=image_folder+str(i)+".png"
img_fname = os.path.basename(img_path)
image = cv2.imread(img_path)






94658560/94653016 [==============================] - 3s 0us/step


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 208, 208, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 208, 208, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 208, 208, 64) 0           bn_conv1[0][0]                   
_____________________________________

In [28]:
%%timeit
yolo.predict(image, float(0.3))

1 loop, best of 3: 397 ms per loop


In [57]:
predict_testdata(1,13068,conf="/content/VRDL_2019/HW3/testing.json",image_folder=image_folder, weights=weights)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 208, 208, 64) 9472        input_8[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 208, 208, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_344 (Activation)     (None, 208, 208, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [58]:
file1 = drive.CreateFile()
file1.SetContentFile('0845058.json')
file1.Upload()

InvalidConfigError: ignored

In [60]:
for i in range(1,13069):
  print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
